In [1]:
from pynq import Overlay
Overlay("base.bit").download()

In [11]:
import math
from pynq.board import Button
from pynq.iop import PMODB
from pynq.iop import PMODA
from pynq.iop import PMOD_GROVE_G4
from pynq.iop import Grove_OLED
from pynq.iop import Grove_IMU
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import requests
import json
import csv
import datetime
import re
import urllib.request
import urllib.parse
import xml.dom.minidom
oled=Grove_OLED(PMODB,PMOD_GROVE_G4)
sense = Grove_IMU(PMODA, PMOD_GROVE_G4)
btns = [Button(index) for index in range(4)]
tlat=0
tlong=0

In [3]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

In [23]:
def getDec(lat,long):
    now = datetime.datetime.now()
    month = now.month
    url="http://www.ngdc.noaa.gov/geomag-web/calculators/calculateDeclination?%s"
    params = urllib.parse.urlencode({'lat1': lat, 'lon1': long, 'resultFormat': 'xml', 'startMonth': month})
    f = urllib.request.urlopen(url % params)
    dom = xml.dom.minidom.parseString(f.read())
    myString = getText(dom.getElementsByTagName("declination")[0].childNodes)
    declination = str(re.findall(r"[-+]?\d*\.\d+|\d+", myString)[0])
    return float(declination)

In [5]:
def getLat():
    url='http://freegeoip.net/csv'
    r=requests.get(url)
    decoded_content = r.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        lat= row[8:9]
        long=row[9:10]
    return float(lat[0])

In [6]:
def getLong():
    url='http://freegeoip.net/csv'
    r=requests.get(url)
    decoded_content = r.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        lat= row[8:9]
        long=row[9:10]
    return float(long[0])

In [7]:
def getBearing(lat,long,targlat,targlong):
    x=math.cos(math.radians(targlat))*math.sin(math.radians(targlong-long))
    y=(math.cos(math.radians(lat))*math.sin(math.radians(targlat)))-(math.sin(math.radians(lat))*math.cos(math.radians(targlat))*math.cos(math.radians(targlong-long)))
    bearing=math.atan2(x,y)
    return math.degrees(bearing)

In [42]:
def screenWrite(targlat,targlong):
    #print('starting')
    #targlat=-10
    #targlong=5
    lat=getLat()
    long=getLong()
    dec=getDec(lat,long)
    bearing=getBearing(lat,long,targlat,targlong)
    sense.reset()
    data=sense.get_compass()
    heading=sense.get_heading()
    altdata=sense.get_accl()
    heading=get_tilt_heading(data, altdata)
    angle=heading+dec+bearing
    angle = (angle+360)%360
    #print(dec)
    #print('check')
    oled.clear()
    #16 characters per line 
    oled.write("lat: "+format(float(lat),'.4f')+"  ")
    if(float(lat)>0):
        oled.write(" ")
    if(abs(float(lat))<10):
        oled.write(" ")
    if(abs(float(lat))<100):
        oled.write(" ")
    oled.write("long: "+format(float(long),'.4f')+" ")
    if(float(long)>0):
        oled.write(" ")
    if(abs(float(long))<10):
        oled.write(" ")
    if(abs(float(long))<100):
        oled.write(" ")
    oled.write("target:         ")
    oled.write("lat: "+format(float(targlat),'.4f')+"  ")
    if(targlat>=0):
        oled.write(" ")
    if(abs(float(targlat))<10):
        oled.write(" ")
    if(abs(float(targlat))<100):
        oled.write(" ")
    oled.write("long: "+format(float(targlong),'.4f')+" ")
    if(targlong>=0):
        oled.write(" ")
    if(abs(float(targlong))<10):
        oled.write(" ")
    if(abs(float(targlong))<100):
        oled.write(" ")
    oled.write("angle: "+format(float(angle),'.5f'))
    #print("oled check")

In [13]:
def getAngleDeg(x,y,mod):
    h=math.sqrt(x*x + y*y)
    rawAng=math.asin(y/h) / (2*math.pi) *360
    if (x>=0 and y>=0):
        adjAng = rawAng
    elif (x<0 and y>=0):
        adjAng = 180-rawAng
    elif (x>=0 and y<0):
        adjAng = 360+rawAng
    elif (x<0 and y<0):
        adjAng = 180-rawAng
    else:
        adjAng = rawAng
        print('error in angle calcs')
        return 0;
    finalAng= adjAng + mod;
    return finalAng
    
def get_tilt_heading(data,altdata):
    [ax, ay, _] = altdata[0:3]
    [mx, my, mz] = data[0:3]
    try:
        pitch = math.asin(-ax)
        roll = math.asin(ay / math.cos(pitch))
    except ZeroDivisionError:
        raise RuntimeError("Value out of range or device not connected")

    xh = mx * math.cos(pitch) + mz * math.sin(pitch)
    yh = mx * math.sin(roll) * math.sin(pitch) + \
    my * math.cos(roll) - mz * math.sin(roll) * math.cos(pitch)
    _ = -mx * math.cos(roll) * math.sin(pitch) + \
    my * math.sin(roll) + mz * math.cos(roll) * math.cos(pitch)
    tilt_heading = 180 * math.atan2(yh, xh) / math.pi
    if yh < 0:
        tilt_heading += 360
    return float("{0:.2f}".format(tilt_heading))

In [41]:
for i in range(0,20):
    screenWrite(float(tlat),float(tlong))

In [32]:
print( math.cos(math.radians(38.627089)) * math.sin(math.radians(4.38101)))

0.05967668697052566


In [11]:
oled.clear()

In [24]:
sense.reset()
data=sense.get_compass()
heading=sense.get_heading()
altdata=sense.get_accl()
heading=get_tilt_heading(data, altdata)
lat=getLat()
long=getLong()
dec=getDec(lat,long)
print(heading+dec)


69.89013
